In [ ]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np 
from glob import glob 
from sklearn.model_selection import train_test_split


# define function to load datasets
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 36)
    return files, targets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load train, test, and validation datasets
data_files, data_targets = load_dataset('/content/drive/MyDrive/ASL_FingerSpelling_Dataset')
# load list of names
names = [item[17:19] for item in sorted(glob("/content/drive/MyDrive/ASL_FingerSpelling_Dataset/*/"))]


X_train, X_test, y_train, y_test = train_test_split(data_files, data_targets, test_size=0.2, random_state=42)

In [ ]:
# print statistics about the dataset

print('There are %s total images.\n' % len(np.hstack([X_train, X_test])))
print('There are %d training images.' % len(X_train))
print('There are %d total categories.' % len(names))
print('There are %d testing images.' % len(X_test)) 


There are 2515 total images.

There are 2012 training images.
There are 36 total categories.
There are 503 testing images.


In [ ]:
# Pre-process the Data
from keras_preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_targets=y_train
test_targets=y_test
train_tensors = paths_to_tensor(X_train).astype('float32')/255  
test_tensors = paths_to_tensor(X_test).astype('float32')/255 

100%|██████████| 503/503 [00:04<00:00, 121.84it/s]


In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3), kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))

#model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(500, activation='relu', kernel_initializer='glorot_normal'))
model.add(Dropout(0.5))
model.add(Dense(36, activation='softmax', kernel_initializer='glorot_normal'))


### TODO: Define your architecture.

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     32896     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 256)       131328    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 28, 28, 256)      0

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
##Model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 
##Model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
##Model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 30

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='/content/sample_data/weights.best.from_scratch_new.hdf5', 
                               monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

history=model.fit(train_tensors, train_targets, validation_split=0.1, epochs=epochs, batch_size=40, callbacks=[checkpointer], verbose=1)

Epoch 1/30
46/46 [==============================] - ETA: 0s - loss: 3.2044 - accuracy: 0.2746
Epoch 1: val_accuracy improved from -inf to 0.68317, saving model to /content/sample_data/weights.best.from_scratch_new.hdf5
46/46 [==============================] - 23s 240ms/step - loss: 3.2044 - accuracy: 0.2746 - val_loss: 1.0619 - val_accuracy: 0.6832
Epoch 2/30
46/46 [==============================] - ETA: 0s - loss: 0.9137 - accuracy: 0.7320
Epoch 2: val_accuracy improved from 0.68317 to 0.79703, saving model to /content/sample_data/weights.best.from_scratch_new.hdf5
46/46 [==============================] - 10s 218ms/step - loss: 0.9137 - accuracy: 0.7320 - val_loss: 0.8358 - val_accuracy: 0.7970
Epoch 3/30
46/46 [==============================] - ETA: 0s - loss: 0.4544 - accuracy: 0.8536
Epoch 3: val_accuracy improved from 0.79703 to 0.86634, saving model to /content/sample_data/weights.best.from_scratch_new.hdf5
46/46 [==============================] - 10s 217ms/step - loss: 0.4544 - 

In [ ]:
model.load_weights('/content/sample_data/weights.best.from_scratch_new.hdf5')

In [ ]:
# get index of predicted dog breed for each image in test set
#dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]
import time
# Start timing
start_time = time.time()
predictions = [model.predict(np.expand_dims(tensor, axis=0))[0] for tensor in test_tensors]
# End timing
end_time = time.time()
print("Time Taken: {} seconds".format(end_time-start_time))

1/1 [==============================] - 0s 23ms/step
Time Taken: 30.40876340866089 seconds


In [ ]:
predicted_classes = np.argmax(np.round(predictions),axis=1)
test_classes = np.argmax(np.round(test_targets),axis=1)
from sklearn.metrics import classification_report
print(classification_report(test_classes, predicted_classes))
from sklearn.metrics import confusion_matrix
confusion_mtx = confusion_matrix(test_classes, predicted_classes)
print(confusion_mtx)

              precision    recall  f1-score   support

           0       0.93      0.88      0.90        16
           1       1.00      0.91      0.95        11
           2       1.00      1.00      1.00        12
           3       1.00      1.00      1.00        12
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00        17
           6       1.00      0.94      0.97        18
           7       1.00      1.00      1.00        13
           8       1.00      1.00      1.00        16
           9       1.00      1.00      1.00        17
          10       1.00      1.00      1.00        19
          11       1.00      1.00      1.00        12
          12       1.00      1.00      1.00        11
          13       0.95      1.00      0.98        20
          14       1.00      1.00      1.00        19
          15       1.00      1.00      1.00        11
          16       0.84      1.00      0.91        21
          17       1.00    